In [1]:
#pandas libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import category_encoders as ce

from sklearn.model_selection import train_test_split
import sklearn.model_selection as modsel
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

%run "../notebooks/cargar_df.ipynb"

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
train = get_train()

In [ ]:
test = get_test()

In [ ]:
train.head().T

In [ ]:
test.head().T

In [ ]:
train = train[train.Stage.isin(['Closed Won', 'Closed Lost'])].copy()
train.shape

In [ ]:
train.Stage.value_counts()

In [ ]:
# Me quedo con los registros con la oportunidad terminada
train.Stage.replace({'Closed Won': 1, 'Closed Lost': 0}, inplace=True)

In [ ]:
train.Stage.value_counts()

In [ ]:
print('Cantidad de opportunity_id de train:',train.Opportunity_ID.nunique())
print('Cantidad de opportunity_id de test:',test.Opportunity_ID.nunique())

In [ ]:
# Junto el set de train y test en un nuevo data frame data
data = pd.concat([train.drop('Stage', axis=1), test])
data

Armo otro data frame con las columnas que no cambian de valor en la misma oportunidad con distintas Ids

In [ ]:
grouped_data = data.groupby('Opportunity_ID').agg(\
    {'Region':'first',
     'Territory':'first',
     'Pricing, Delivery_Terms_Quote_Appr':'first',
     'Pricing, Delivery_Terms_Approved':'first',
     'Bureaucratic_Code_0_Approval':'first',
     'Bureaucratic_Code_0_Approved':'first',
     'Bureaucratic_Code':'first',
     'Account_Created_Date':'first',
     'Source ':'first',
     'Billing_Country':'first',
     'Account_Name':'first',
     'Opportunity_Name':'first',
     'Account_Owner':'first',
     'Opportunity_Owner':'first',
     'Account_Type':'first',
     'Opportunity_Type':'first',
     'Quote_Type':'first',
     'Delivery_Terms':'first',
     'Opportunity_Created_Date':'first',
     'Brand':'first',
     'Product_Type':'first',
     'Size':'first',
     'Product_Category_B':'first',
     'Price':'first',
     'Currency':'first',
     'Quote_Expiry_Date':'first',
     'Last_Modified_Date':'first',
     'Last_Modified_By':'first',
     'ASP_Currency':'first',
     'Total_Amount_Currency':'first',
     'Total_Taxable_Amount_Currency':'first',
     'Total_Taxable_Amount':'first'}).reset_index()
grouped_data

In [ ]:
X = pd.DataFrame({'Opportunity_ID':grouped_data.Opportunity_ID})

## Agrego Features

In [ ]:
X['Pricing, Delivery_Terms_Quote_Appr'] = grouped_data['Pricing, Delivery_Terms_Quote_Appr']

In [ ]:
X['Pricing, Delivery_Terms_Approved'] = grouped_data['Pricing, Delivery_Terms_Approved']

In [ ]:
pricing_ok = grouped_data.agg(lambda x: int(x['Pricing, Delivery_Terms_Quote_Appr'] \
              == x['Pricing, Delivery_Terms_Approved']), axis = 1)
pricing_ok 

In [ ]:
# Binaria, es 1 si se necesitaba aprobación y se aprobó o si no la necesitaba
# Es 0 si necesitaba aprobacion y no se aprobo
X['pricing_aprobada'] = pricing_ok

X.head()

Agrego features binarios Bureaucratic_Code

In [ ]:
X['Bureaucratic_Code_0_Approval'] = grouped_data['Bureaucratic_Code_0_Approval']

In [ ]:
X['Bureaucratic_Code_0_Approved'] = grouped_data['Bureaucratic_Code_0_Approved']

In [ ]:
bureaucratic_ok = grouped_data.agg(lambda x: int(x['Bureaucratic_Code_0_Approval'] \
              == x['Bureaucratic_Code_0_Approved']), axis = 1)
bureaucratic_ok 

In [ ]:
# Binaria, es 1 si se necesitaba aprobación y se aprobó o si no la necesitaba
# Si necesitaba aprobacion y no se aprobo es 0
X['bureaucratic_aprobada'] = bureaucratic_ok

X.head()

Feature: binaria 1 si hay mas de 1 ID de la oportunidad, 0 si hay solo un registro

In [ ]:
ids = data.groupby('Opportunity_ID').ID.count()
ids

In [ ]:
group1 = (ids != 1).astype('int8').reset_index()
group1

In [ ]:
X['amount_ids'] = group1.ID
X.head()

Feature: menor anio de creacion de oportunidad 

In [ ]:
data.groupby('Opportunity_ID').Delivery_Year.min()

In [ ]:
min_year = data.groupby('Opportunity_ID').Delivery_Year.min().reset_index()

In [ ]:
X['min_year'] = min_year.Delivery_Year
X.head()

Feature: cantidad de dias activos

In [ ]:
# Calculo la cantidad de dias activos que tuvo cada oportunidad
days = grouped_data['Last_Modified_Date'] - grouped_data['Opportunity_Created_Date']

In [ ]:
days.dt.days

In [ ]:
X['active_days'] = days.dt.days
X

In [ ]:
# Me fijo cuantos null hay por columnas
print('Cantidad de nulos por columna')
grouped_data.replace({'None':np.NaN}).isna().sum()

Paso las columnas con muchos nulos a binario:
    1 si tiene valor y 0 si es nulo

In [ ]:
X['has_source'] = (grouped_data['Source '] == 'None').astype('int8')
X['has_brand'] = (grouped_data['Brand'] == 'None').astype('int8')
X['has_product_Type'] = (grouped_data['Product_Type'] == 'None').astype('int8')
X['has_size'] = (grouped_data['Size'] == 'None').astype('int8')
X['has_product_Category_B'] = (grouped_data['Product_Category_B'] == 'None').astype('int8')
X['has_price'] = (grouped_data['Price'] == 'None').astype('int8')
X['has_currency'] = (grouped_data['Currency'] == 'None').astype('int8')
X

In [ ]:
# Agrego feature, el numero de delivery quarter

first_quarter = data.groupby('Opportunity_ID').Delivery_Quarter.first()\
                     .agg(lambda x: x[1:2]).astype('int8')\
                     .reset_index().drop('Opportunity_ID', axis=1)

X = pd.concat([X, first_quarter], axis = 1, sort=False)
X.head()

Features: menor mes y anio de la columna Month

In [ ]:
data['month'] = data.Month.dt.month
data['year'] = data.Month.dt.year

In [ ]:
u = data.groupby('Opportunity_ID').agg({'month':'min', 'year':'min'})\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X, u], axis = 1, sort=False)
X

In [ ]:
grouped_data['Territory'].dtype

In [ ]:
# Transformo las columnas categoricas con binary encoder

categorical_cols = []

for col in grouped_data.columns:
    if (grouped_data[col].dtype == 'O'):
        categorical_cols.append(col)
        print(col)

print()
print(categorical_cols)

In [ ]:
for col in categorical_cols:
    X[col] = grouped_data[col]

encoder = ce.BinaryEncoder(cols = categorical_cols)
X = encoder.fit_transform(X)

X

In [ ]:
# Elimino los outliers de las columnas ASP y ASP_Converted
data_asp_mean = data[data.ASP <= 10].ASP.mean()
outliers =  data.ASP <= 10
data.ASP = data.ASP.where(outliers, data_asp_mean)

data_asp_mean = data[data['ASP_(converted)'] <= 10]['ASP_(converted)'].mean()
outliers =  data['ASP_(converted)'] <= 10
data['ASP_(converted)'] = data['ASP_(converted)'].where(outliers, data_asp_mean)


# Del Tp1

# https://data.oecd.org/conversion/exchange-rates.htm
price_usd_convertion_by_year = {
    'EUR': {2016: 0.940, 2017: 0.923, 2018: 0.848, 2019: 0.893, 2020: 0.860},
    'JPY': {2016: 113.138, 2017: 116.667, 2018: 110.424, 2019: 109.008, 2020: 104.66},
    'AUD': {2016: 1.400, 2017: 1.358, 2018: 1.340, 2019: 1.439, 2020: 1.420},
    'GBP': {2016: 0.770, 2017: 0.808, 2018: 0.750, 2019: 0.784, 2020: 0.770},
    'USD': {2016: 1, 2017: 1, 2018: 1, 2019: 1, 2020: 1}
}

def get_convertion_usd(x):
    currency = x['Total_Amount_Currency']
    year = pd.to_datetime(x['Month']).year
    return price_usd_convertion_by_year[currency][year]
    
data['Total_Amount_USD'] = data['Total_Amount'] / data.apply(lambda x: get_convertion_usd(x), axis=1)

data['Total_Amount_USD'].head()

In [ ]:
# Agrego como feature el promedio de asp
u = data.groupby('Opportunity_ID').ASP.mean()\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X,u], axis=1, sort=False)

In [ ]:
# Agrego como feature el promedio de asp converted
u = data.groupby('Opportunity_ID')['ASP_(converted)'].mean()\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X,u], axis=1, sort=False)

In [ ]:
# Agrego como feature el promedio de Total_Amount_USD
u = data.groupby('Opportunity_ID')['Total_Amount_USD'].mean()\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X,u], axis=1, sort=False)
X

In [ ]:
X.isna().sum()

In [ ]:
total_mean = X.Total_Amount_USD.mean()
total_mean

In [ ]:
X = X.replace({np.NaN: total_mean})

## Prediccion

In [ ]:
X.info()

In [ ]:
train_ids = train.Opportunity_ID.unique()
trainX = X[X['Opportunity_ID'].isin(train_ids)]
trainX

In [ ]:
y = train.groupby('Opportunity_ID')['Stage'].min()
y

In [ ]:
(y.index != trainX.Opportunity_ID).sum()

In [ ]:
test_ids = test.Opportunity_ID.unique()
testX = X[X['Opportunity_ID'].isin(test_ids)]
testX

In [ ]:
pd.merge(trainX, y, on='Opportunity_ID').to_csv(\
    '../files_csv/train_173_features_bin_encoding', index=False)

In [ ]:
testX.to_csv('../files_csv/test_173_features_bin_encoding', index=False)

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(trainX.drop(['Opportunity_ID'], axis=1), y, test_size=0.2, random_state=123)

In [ ]:
# Calculo el error
X_train, X_test, y_train, y_test = \
    train_test_split(trainX, y, test_size=0.2, random_state=123)

xgb_reg = xgb.XGBRegressor(objective = 'binary:logistic', \
                           colsample_bytree = 0.3, \
                           learning_rate = 0.1, \
                           max_depth = 5, \
                           alpha = 10, \
                           n_estimators = 10, \
                           eval_metric='logloss')
xgb_reg.fit(X_train, y_train)
preds = xgb_reg.predict(X_test)
ll = log_loss(y_test, preds, normalize=True)
print("Log Loss: %f" % (ll))

In [ ]:
# Feature importances
xgb_reg.feature_importances_

In [ ]:
p = pd.DataFrame({'feature':X_train.columns, 'value':xgb_reg.feature_importances_})

In [ ]:
df = p.sort_values('value').nlargest(50, 'value')

plt.bar(df.feature, df.value, )
plt.xlabel('Features')
plt.xticks(rotation=90)
plt.ylabel('Importancia')
plt.title('Importancia Features con XGBRegressor')
plt.show()

In [ ]:
df = p.sort_values('value').nsmallest(X.shape[1]-50, 'value')

plt.bar(df.feature, df.value)
plt.xlabel('Features')
plt.xticks(rotation=90)
plt.ylabel('Importancia')
plt.title('Importancia Features con XGBRegressor')
plt.show()

In [ ]:
rf_model = RandomForestRegressor(random_state=1, n_estimators=5)


rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

loss = log_loss(y_test, preds)
print("Loss: %f" % (loss))

In [ ]:
tree0 = rf_model.estimators_[0]

In [ ]:
p = pd.DataFrame({'feature':X_train.columns, 'value':tree0.feature_importances_})
p = p.sort_values('value')

In [ ]:
plt.bar(p.feature, p.value)
plt.xlabel('Features')
plt.xticks(rotation=90)
plt.ylabel('Importancia')
plt.title('Importancia Features con Random Forest')
plt.show()

In [ ]:
rf_model = RandomForestRegressor(random_state=1, n_estimators=5)

rf_model.fit(trainX, y)
preds = rf_model.predict(testX)

In [ ]:
preds.shape

In [ ]:
prediccion = pd.DataFrame({'Opportunity_ID':testX.Opportunity_ID, 'Target':preds})
prediccion

In [ ]:
prediccion.to_csv('prediccion_random_forest_2', index=False)

In [ ]:
# Puede tardar mucho, los valores que me habian salido para menos features fueron:
# colsample_bytree = 0.5, 
# learning_rate = 0.1, 
# max_depth = 12, 
# alpha = 5, 
# n_estimators = 50, 

# Set the parameters by cross-validation
tuned_parameters = [{'colsample_bytree': [0.1,0.3,0.5],
                     'learning_rate': [0.1, 0.05],
                     'max_depth': [5,8,10,12],
                     'alpha': [5,10,15],
                     'n_estimators': [10,50],
                     'objective': ['binary:logistic'],
                     'eval_metric': ['logloss']}]

score = 'log_loss'
print('# Tuning hyper-parameters for XGBoost and Log loss')

X_train, X_test, y_train, y_test = \
    train_test_split(trainX, y, test_size=0.2, random_state=123)

xgb_reg = xgb.XGBRegressor()
clf = GridSearchCV(\
    xgb_reg, tuned_parameters, cv=5)
clf.fit(X_train, y_train)

print("Best Params:")
print(clf.best_params_ )
print()
print('Best Score:')
print(clf.best_score_)

In [ ]:
# Calculo el error
X_train, X_test, y_train, y_test = \
    train_test_split(trainX, y, test_size=0.15, random_state=123)

xgb_reg = xgb.XGBRegressor(objective = 'binary:logistic', \
                           colsample_bytree = 0.5, \
                           learning_rate = 0.1, \
                           max_depth = 12, \
                           alpha = 5, \
                           n_estimators = 50, \
                           subsample=0.5,\
                           eval_metric='logloss')
xgb_reg.fit(X_train, y_train)
preds = xgb_reg.predict(X_test)
ll = log_loss(y_test, preds, normalize=True)
print("Log Loss: %f" % (ll))

In [ ]:
p = pd.DataFrame({'feature':X_train.columns, 'value':xgb_reg.feature_importances_})
p = p.sort_values('value')

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(p.feature, p.value)
plt.xlabel('Features')
plt.xticks(rotation=90)
plt.ylabel('Importancia')
plt.title('Importancia Features con XGBRegressor')
plt.show()

In [ ]:
xgb_reg = xgb.XGBRegressor(objective = 'binary:logistic', \
                           colsample_bytree = 0.5, \
                           learning_rate = 0.1, \
                           max_depth = 12, \
                           alpha = 5, \
                           n_estimators = 50, \
                           eval_metric='logloss')

xgb_reg.fit(trainX, y)
preds = xgb_reg.predict(testX)

In [ ]:
preds.shape

In [ ]:
prediccion = pd.DataFrame({'Opportunity_ID':testX.Opportunity_ID, 'Target':preds})
prediccion

In [ ]:
prediccion.to_csv('prediccion_xgboost_7', index=False)